In [0]:
!pip install tensorflow==1.12.0
# TensorFlow and tf.keras
# its just working with tensorflow 1.13.1, with others has problem


    100% |████████████████████████████████| 83.1MB 359kB/s 
    100% |████████████████████████████████| 3.1MB 7.2MB/s 
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1


In [0]:
import tensorflow as tf
print(tf.__version__)

1.13.1


In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [0]:
!ls "/content/gdrive/My Drive/NLPHW1"
root_path = '/content/gdrive/My Drive/NLPHW1/'
print(root_path)

 cityu_best_models		      logging
'Copy of LastCode_NLPHW1 (2).ipynb'   msr_best_models
 Datasets			      pku_best_models
 embedding			      tf1.12.0TestPrediction.ipynb
'LastCode_NLPHW1 (2).ipynb'
/content/gdrive/My Drive/NLPHW1/


**imports and basic definitions**

In [0]:
import os
import re
import errno
import pickle
import numpy as np
from collections import Counter
import codecs
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import crf
   
#from hanziconv import HanziConv

START_SENT = "گ"
END_SENT = "چ"
UNK = "ژ"
PAD = "پ"

TAGB,TAGI,TAGE,TAGS=0,1,2,3

rNUM = '(-|\+)?\d+((\.|·)\d+)?%?'
rENG = '[A-Za-z_.]+'

def make_sure_path_exists(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise


In [0]:
def remove_spaces(src_file, des_file, encode='UTF-8'):
    with open(src_file, encoding=encode) as src_file, open(des_file, 'w', encoding=encode) as des_file:
        for line in src_file: 
            # cleaning input                     
            sent_words = line.split()                             
            sentence = ''.join(sent_words) + '\n'
            des_file.write(sentence)     

# Function for cleaning file, make charachters better and spaces standard space.
def clean_data(src_file, des_file, encode='UTF-8'):
    with open(src_file, encoding=encode) as src_file, open(des_file, 'w', encoding=encode) as des_file:
        sentences=[]
        for line in src_file: 
            # cleaning input
            rstring = ""
            for uchar in line:
                inside_code = ord(uchar)
                if inside_code == 12288:
                    inside_code = 32
                elif 65281 <= inside_code <= 65374:
                    inside_code -= 65248
            
                rstring += chr(inside_code)      
            
            sent_words = rstring.split()
            
            new_sent_words = []
            
            for word in sent_words:
                word = re.sub('\s+', '', word, flags=re.U)    
                new_sent_words.append(word)
            
            sentences.append(new_sent_words)
        for sent_num in range(len(sentences)):            
#             if(sent_num != len(sentences)-1):
            des_file.write(' '.join(sentences[sent_num]) + '\n') 
#             else:
#                 des_file.write(' '.join(sentences[sent_num])) 

# getting test data function with unigram, bigram and trigrams
def get_test_data(filename,ngrams2id, usebigram, usetrigram,usefourgram):   
    x,y=[],[]
    with codecs.open(filename,'r','utf-8') as f:
        # for every line we split and then tag every word with proper id.
        for line in f:   
            line_x = []
            # for every line we remove spaces and then assume every 5 (2 left char and 2 right char)
            # char as a new context, for every context we also calculate bigram and trigram and add them too
            line=re.sub(u'\s+','',line.strip())
            contexs=window(line)
            for contex in contexs:
                charx=[]
                #contex window
                charx.extend([ngrams2id.get(c,ngrams2id[UNK]) for c in contex])
                #bigram feature
                if usebigram:
                    charx.extend([ngrams2id.get(bigram, ngrams2id[UNK]) for bigram in ngram(contex,2)])
                if usetrigram:
                    charx.extend([ngrams2id.get(trigram, ngrams2id[UNK]) for trigram in ngram(contex,3)])
                if usefourgram:
                    charx.extend([ngrams2id.get(fourgram, ngrams2id[UNK]) for fourgram in ngram(contex,4)])
                line_x.append(charx)
            x.append(line_x)
    return x

# getting ngrams function
def ngram(ustr,n=2):
    ngram_list=[]
    for i in range(len(ustr)-n+1):
        ngram_list.append(ustr[i:i+n])
    return ngram_list


# window return sentence as list of 5 length context, for every word we are 
# gettnig 4 neighbour, 2 left and 2 right for having more training data.
def window(ustr,left=2,right=2):
    sent=''
    for i in range(left):
        sent+=START_SENT
    sent+=ustr
    for i in range(right):
        sent+=END_SENT
    windows=[]
    for i in range(len(ustr)):
        windows.append(sent[i:i+left+right+1])
    return windows



In [0]:
def prepare_files_test(all_datasets):    
    remove_spaces(root_path+'Datasets/gold/{}_test_gold.utf8'.format(all_datasets), root_path+'Datasets/gold/no_space_{}_test_gold.utf8'.format(all_datasets))

    clean_data(root_path+'Datasets/gold/no_space_{}_test_gold.utf8'.format(all_datasets), root_path+'Datasets/cleaned_data/{}/raw/cleaned_no_space_test.utf8'.format(all_datasets))


In [0]:
all_datasets = 'pku'
prepare_files_test(all_datasets)

In [0]:
# use pretrained embeddings function, for chars we are using pretrain file and for bigram, trigram 
# and fourgram we are using mean of embeddings of all unigrams of them
def reading_pretrained_embeddings(filename, ngrams2id):
    # Reading Pretrained Embeddings from file
    pretrain_embeddigs = {}
    with codecs.open(filename, "r", "utf-8") as f:
        for line in f:
            pre_train = line.split()
            if len(pre_train) > 2:
                word = pre_train[0]
                if word in ngrams2id:
                    vec = pre_train[1:]
                    pretrain_embeddigs[word] = vec                
    
    print("pretraine embedding files reading finished ...")
    # making embeddings for all ngrams2id.
    embedding_dim = len(next(iter(pretrain_embeddigs.values())))
    out_of_vocab = 0
    out = np.ones((len(ngrams2id), embedding_dim))
    for ngram in ngrams2id.keys():
        if len(ngram) == 1:
            if ngram in pretrain_embeddigs.keys():        
                out[ngrams2id[ngram]]=np.array(pretrain_embeddigs[ngram])
            else:
                out_of_vocab+=1
                np.random.uniform(-0.8, 0.8, embedding_dim)
    for ngram in ngrams2id.keys():        
        #embedding for bigrams
        if len(ngram) == 2:            
            out[ngrams2id[ngram]]= (out[ngrams2id[ngram[0]]]+out[ngrams2id[ngram[1]]])/2
        #embedding for trigrams
        if len(ngram) == 3:
            out[ngrams2id[ngram]]= (out[ngrams2id[ngram[0]]]+out[ngrams2id[ngram[1]]]+out[ngrams2id[ngram[2]]])/3
        #embedding for fourgrams
        if len(ngram) == 4:
            out[ngrams2id[ngram]]= (out[ngrams2id[ngram[0]]]+out[ngrams2id[ngram[1]]]+out[ngrams2id[ngram[2]]]+out[ngrams2id[ngram[3]]])/4               
    print('out_of_vocab characters: %d' % (out_of_vocab) )         
    return out,out_of_vocab


In [0]:
# Retriev word_embeddings and PAD word id from Saved pkl File
def ngrams_pretrain_embed():
    FILE_FOR_NGRAM = 'All_datasets'
    ngrams2id = pickle.load(open(root_path+'Datasets/output_{}_ngram2id.pkl'.format(FILE_FOR_NGRAM), "rb"))["ngrams2id"]    
    
    # get ngrams2ids's pretrained embeddings
    word_embeddings, out_of_vocab = reading_pretrained_embeddings(root_path+'embedding/character.vec', ngrams2id)
    PAD_ID = ngrams2id[PAD]
    return ngrams2id,word_embeddings,PAD_ID

In [0]:
ngrams2id,word_embeddings,PAD_ID = ngrams_pretrain_embed()
print(len(word_embeddings))


pretraine embedding files reading finished ...
out_of_vocab characters: 380
1773043


In [0]:
#Retriev train, test data from pkl files
FILE_NAME = 'pku'

TRAIN_FILE_ADD = root_path+'Datasets/output_{}_train_test.pkl'.format(FILE_NAME)

train_test_file = pickle.load(open(TRAIN_FILE_ADD, "rb"))

subset_train = 1000 # Put number of subset data you want for train
subset_dev= 500
subset_test= 1000

test_x = train_test_file["test_x"][slice(0, subset_test)]
test_y = train_test_file["test_y"][slice(0, subset_test)]

PAD_ID = ngrams2id[PAD]


# all_datasets = 'pku'
# TEST_FILENAME = root_path+'Datasets/cleaned_data/{}/raw/cleaned_no_space_test.utf8'.format(all_datasets) # Address of no sace test file.
# test_x = get_test_data(TEST_FILENAME, ngrams2id, usebigram=True, usetrigram = True,usefourgram = True)#[slice(0, subset_test)]

print("size of test data ....")
print(len(test_x))
print(len(test_x[0]))
print(len(test_x[0][0]))

print(len(test_y))
print(len(test_y[0]))

size of test data ....
1000
21
14
1000
21


In [0]:
# ===-----------------------------------------------------------------------===
# Trainig Section
# ===-----------------------------------------------------------------------===

def padding(X,padding_word):
	max_len = 0
	for x in X:
		if len(x) > max_len:
			max_len = len(x)
	padded_X = np.ones((len(X), max_len), dtype=np.int32) * padding_word
	for i in range(len(X)):
		for j in range(len(X[i])):
			padded_X[i, j] = X[i][j]
	return padded_X

def padding3(X,padding_word):
	max_len = 0
	for x in X:
		if len(x) > max_len:
			max_len = len(x)
	padded_X = np.ones((len(X), max_len,14), dtype=np.int32) * padding_word
	for i in range(len(X)):
		for j in range(len(X[i])):
			padded_X[i, j] = X[i][j]
	return padded_X

# function for calculating umber of hits and number of all predictions.
def number_of_batch_hits(y_batch_pred,y_batch_true):
    num_hits = 0
    num_all_chars = 0    
    for i in range(len(y_batch_true)):
     #   print("---------------------")
        for j in range(len(y_batch_true[i])):
            num_all_chars = num_all_chars+1
            if y_batch_pred[i][j] == y_batch_true[i][j]:
               num_hits = num_hits+1    
    return num_hits, num_all_chars

# ----------------- Add Summary Function ------------------------------------------
def add_summary(writer, name, value, global_step):
    summary = tf.Summary(value=[tf.Summary.Value(tag=name, simple_value=value)])
    writer.add_summary(summary, global_step=global_step)


In [0]:

VOCAB_SIZE =  word_embeddings.shape[0]
WORD_EMBEDDING_DIM = word_embeddings.shape[1]

print(WORD_EMBEDDING_DIM)
print(VOCAB_SIZE)

BATCH_SIZE = 16
HIDDEN_LAYER_DIM = 128
LEARNING_RATE = 0.01
NUM_CLASSES = 4
L2_REGU_LAMBDA=0.0001
NUM_LAYERS = 1
CLIP=5

100
1773043


In [0]:
# --------------------- Tensorflow part ---------------------------------------------------

def create_tensorflow_model(vocab_size, embedding_size, hidden_layer_dim):
    print("Creating TENSORFLOW model")
     
    # Inputs have (batch_size, timesteps) shape.
    X = tf.placeholder(dtype=tf.int32,shape=[None,None,14],name='input_x')   
    # Labels have (batch_size,) shape.
    labels = tf.placeholder(dtype=tf.int32,shape=[None,None],name='input_y')
    # dropout_keep_prob is a scalar.
    dropout_keep_prob=tf.placeholder(dtype=tf.float32,name='dropout_keep_prob')
    # Calculate sequence lengths to mask out the paddings later on.
    seq_length = tf.reduce_sum(tf.cast( tf.not_equal(X[:,:,2], tf.ones_like(X[:,:,2]) * PAD_ID), tf.int32), 1)
        
    with tf.variable_scope('embeddings', reuse=tf.AUTO_REUSE):
        embedding_matrix = tf.Variable(word_embeddings, dtype=tf.float32, name='embedding')
        #embedding_matrix = tf.get_variable("embeddings", shape=[vocab_size, embedding_size])
        embeddings = tf.nn.embedding_lookup(embedding_matrix, X)
        embeddings = tf.reshape(embeddings,[tf.size(X[:,1,1]),-1,14*embedding_size])
    
    # embeddings shape (batch size, sentence length with padding, 1400)
    embeddings=tf.nn.dropout(tf.cast(embeddings,tf.float32),dropout_keep_prob)
    
    with tf.variable_scope('rnn_cell', reuse=tf.AUTO_REUSE):
            print ('rnn_cell is lstm')
            def lstm_cell():
                return tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(hidden_layer_dim), output_keep_prob=dropout_keep_prob)
            
            def gru_cell():
                return tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.GRUCell(hidden_layer_dim), output_keep_prob=dropout_keep_prob)

            
            stacked_fw_lstm = tf.nn.rnn_cell.MultiRNNCell(
                [gru_cell() for _ in range(NUM_LAYERS)])
            
            stacked_bw_lstm = tf.nn.rnn_cell.MultiRNNCell(
                [lstm_cell() for _ in range(NUM_LAYERS)])
        
    with tf.variable_scope('rnn', reuse=tf.AUTO_REUSE):                        
            (forward_output,backword_output),_=tf.nn.bidirectional_dynamic_rnn(
                cell_fw=stacked_fw_lstm,
                cell_bw=stacked_bw_lstm,
                inputs=embeddings,
                sequence_length=seq_length,
                dtype=tf.float32
            )
            outputBidirection=tf.concat([forward_output,backword_output],axis=2)
            print ('outputBidirection is ok')    
            
    print ('Loss Starts ....')
    with tf.variable_scope('loss', reuse=tf.AUTO_REUSE):
        output=layers.fully_connected(
            inputs=outputBidirection,
            num_outputs=NUM_CLASSES,
            activation_fn=None            
            )
        print ('output is ok ....')
        #crf
        log_likelihood, transition_params = crf.crf_log_likelihood(
            output, labels, seq_length)
        print ('crf_log_likelihood is ok ....')
        
        loss = tf.reduce_mean(-log_likelihood)
        print ('loss crf_log_likelihood is ok ....')
    
    print ('train-op Starts ....')
    with tf.variable_scope('train_op', reuse=tf.AUTO_REUSE):
        optimizer=tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
        print ('optimizer is ok ....')
    
        tvars=tf.trainable_variables()
        print ('tvars is ok ....')
    
        l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tvars if v.get_shape().ndims > 1])
        print ('l2_loss is ok ....')
        
        loss=loss+L2_REGU_LAMBDA*l2_loss
        print ('loss L2 is ok ....')
        
        grads,_=tf.clip_by_global_norm(tf.gradients(loss,tvars),CLIP)
        print ('grads is ok ....')
        
        train_op=optimizer.apply_gradients(zip(grads,tvars))
        print ('train_op apply_gradients is ok ....')
               
    return X, labels, output, train_op, dropout_keep_prob, loss, transition_params, seq_length


In [0]:
# create tensorflow model
X, labels, output, train_op, dropout_keep_prob, loss, transition_params, seq_length = create_tensorflow_model(VOCAB_SIZE, WORD_EMBEDDING_DIM, HIDDEN_LAYER_DIM)


Creating TENSORFLOW model
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
rnn_cell is lstm
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
outputBidirection is ok
Loss Starts ....
output is ok ....
crf_log_likelihood is ok ....
loss crf_log_likelih

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


grads is ok ....
train_op apply_gradients is ok ....


In [0]:
#Restore Saved Tensorflow model.
import tensorflow as tf
saver = tf.train.Saver()

with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph( root_path+'best_model_path85.25236692748163/model.ckpt.meta')
    new_saver.restore(sess, root_path+'best_model_path85.25236692748163/model.ckpt')
    #saver.restore(sess, root_path+'best_model_path/model.ckpt')
    print("Model restored.")
    
    #for tensor in tf.get_default_graph().get_operations():
     #   print(tensor.name)
    
    test_hit_all=0
    test_all_char=0        
    test_pred = []
    for i in range(0, len(test_x), BATCH_SIZE):
        batch_x = test_x[slice(i, i + BATCH_SIZE)]
        batch_x = padding3(batch_x, PAD_ID)

        lengths,unary_scores,transition_param_other = sess.run(
            [seq_length,output,transition_params], feed_dict = {X:batch_x, dropout_keep_prob:1.0})
        predict=[]
        for unary_score,length in zip(unary_scores,lengths):
            if length > 0 :
                viterbi_sequence, _=crf.viterbi_decode(unary_score[:length],transition_param_other)
                predict.append(viterbi_sequence)
            else:
                predict.append("")
            #print('sentence length : %d' % (length))
            #print(viterbi_sequence)
        test_pred += predict          
        
        test_batch_hit,test_batch_char = number_of_batch_hits(predict, test_y[slice(i, i + BATCH_SIZE)])            
        print('test_batch_hit:%f test_batch_char:%f batch_accuracy:%f' % (test_batch_hit, test_batch_char,(test_batch_hit/test_batch_char)*100 ))            
        test_hit_all+=test_batch_hit
        test_all_char+=test_batch_char

    print('test_hit:%f test_all_char:%f accuracy:%f' % (test_hit_all,test_all_char,(test_hit_all/test_all_char)*100 ))
    

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/NLPHW1/best_model_path85.25236692748163/model.ckpt
Model restored.


FailedPreconditionError: ignored

In [0]:
def ids_to_tag_file(test_pred,output_file):
    with codecs.open(output_file, 'w','utf-8') as f:
        for i in range(len(test_pred)):
            tag_list = ''
            for j in range(len(test_pred[i])):
                if test_pred[i][j]==0:
                    tag_list +='B'
                elif test_pred[i][j]==1:
                    tag_list+='I'
                elif test_pred[i][j]==2:
                    tag_list+='E'
                elif test_pred[i][j]==3:
                    tag_list+='S'
            #print(tag_list)
            if(i != len(test_pred)-1):
                f.write(''.join(tag_list).strip())            
                f.write('\n') 
            else:
                f.write(''.join(tag_list).strip())            
    print("result is ready ...")
   


In [0]:
ids_to_tag_file(test_pred,output_file=root_path+'Datasets/cleaned_data/pku/bies/result.utf8')


result is ready ...


In [0]:
ALL_TAGS = {"B", "I", "E", "S"}

def is_valid_prediction(prediction_iter, gold_iter):
    assert len(prediction_iter) == len(gold_iter), "Prediction and gold have different lengths"

    prediction_tags = set()
    gold_tags = set()
    nr_line = 1
    for preds, gold in zip(prediction_iter, gold_iter):
        assert len(preds) == len(gold), "Line " + str(nr_line) + ": lengths mismatch"
        prediction_tags.update(preds)
        gold_tags.update(gold)
        nr_line += 1
    
    prediction_tags = {t.upper() for t in prediction_tags}
    gold_tags = {t.upper() for t in gold_tags}
    
    assert len(gold_tags.difference(ALL_TAGS)) == 0, "Unknown tag detected in gold"
    assert len(prediction_tags.difference(ALL_TAGS)) == 0, "Unknown tag detected in predictions"


def score(prediction_iter, gold_iter, verbose=False):
    """
    Returns the precision of the model's predictions w.r.t. the gold standard (i.e. the tags of the
    correct word segmentation).

    :param prediction_iter: List of strings in the BIES format representing the model's predictions.
    :param gold_iter: List of strings in the BIES format representing the gold standard.

    :return: precision [0.0, 1.0]
    
    Ex. predictions_iter = ["BEBESBIIE",
                            "BIIIEBEBESS"]
        gold_iter = ["BEBIEBIES",
                     "BIIESBEBESS"]
        output: 0.7
    
    The same result can be obtain by passing list of lists
    Ex. predictions_iter = [["B", "E", "B", "E", "S", "B", "I", "I", "E"],
                            ["B", "I", "I", "I", "E", "B", "E", "B", "E", "S", "S"]]
        gold_iter = [["B", "E", "B", "I", "E", "B", "I", "E", "S"],
                     ["B", "I", "I", "E", "S", "B", "E", "B", "E", "S", "S"]]
        output: 0.7

    
    """
    
    is_valid_prediction(prediction_iter, gold_iter)

    right_predictions = 0
    wrong_predictions = 0

    for prediction_sentence, gold_sentence in zip(prediction_iter, gold_iter):
        for prediction_tag, gold_tag in zip(prediction_sentence, gold_sentence):
            if prediction_tag == gold_tag:
                right_predictions += 1
            else:
                wrong_predictions += 1

    precision = right_predictions / (right_predictions + wrong_predictions)
    if verbose:
        print("Precision:\t", precision)

    return precision


def label_text_to_iter(file_path):
    iter_ = []
    with open(file_path) as f:
        for line in f:
            line = line.strip().upper()
            iter_.append(line)
    return iter_





In [0]:
prediction_file = root_path+'Datasets/cleaned_data/pku/bies/result.utf8'
gold_file = root_path+'Datasets/cleaned_data/pku/bies/test.utf8'

prediction_iter = []
score(label_text_to_iter(prediction_file), label_text_to_iter(gold_file), verbose=True)


AssertionError: ignored